<a href="https://colab.research.google.com/github/SolanaO/Knowledge_Graphs_Assortment/blob/master/arXiv_KG/2_ArXiv_KG_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Knowledge Graph on ArXiv Dataset


## Description

In this notebook, we use the enhanced arxiv data prepared in `1.ArXiv_Data_Enhancer.ipynb` to create several files (or dataframes only) that will be used to upload the data into a Neo4j graph database. This notebook does not require any special settings, as we will only perform data wrangling and connect to a Neo4j instance.

We will create a graph with node labels Article, Author, Categories, UpdateDate, DOI, Journal, Keyword and Topic. Each of these features needs to be collected as a list of distinct elements with unique identifier to be uploaded in the graph. Since Neo4j does not get along with missing values, we will replace them with various strings.

## Colab Setup

In [ ]:
# Load and mount the drive helper
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive')

# Set the working directory
%cd '/content/drive/MyDrive/arxivKG/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/arxivKG


In [ ]:
# Create a path variable for the data folder
data_path = '/content/drive/MyDrive/arxivKG/datas/'

### Files created in notebook 1

# Articles with keywords/keyphrases
parsed_keys_file = 'parsed_math_keys.csv'

# Descriptions and labels for keywords clusters
descriptions_labels_keys_file = 'descriptions_labels.csv'

### Files created in notebook 2

# Articles for KG file
articles_file = "articles_parsed.csv"

# Authors for KG file
authors_file = "authors_parsed.csv"

# Relations: authors & articles
articles_authors_relations_file = "articles_authors_relations.csv"

# Categories for KG file
categories_file = "categories_parsed.csv"

# DOI for KG file
doi_file = "doi_parsed.csv"

# UpdateDate for KG file
udates_file = "updated_date_parsed.csv"

# Journals for KG file
journals_parsed = "journals_parsed.csv"

# Parsed keywords for KG file
keywords_file = "keywords_parsed.csv"

# Reports file
reports_parsed = "reports_parsed.csv"

In [ ]:
import pandas as pd
#pd.set_option('display.max_colwidth', 60)

import hashlib
import ast

import re

In [ ]:
# Add hashing functions - choose one

def hash_text(text, length):
    """Use full length for larger datasets to avoid collisions."""
    full_hash = hashlib.sha256(str(text).encode('utf-8')).hexdigest()
    return full_hash[:length]


def hash_text_md5(text):
    """Shorter hashes for smaller datasets."""
    return hashlib.md5(str(text).encode('utf-8')).hexdigest()

import base64
def hash_text_base64(text):
    """Longer hashes in a compact output."""
    hash_bytes = hashlib.sha256(str(text).encode('utf-8')).digest()
    return base64.b64encode(hash_bytes).decode('utf-8')[:8]

## Graph Data Preparation

### Parse Keywords

In [ ]:
# Keywords, soft labels/clusters, descriptions and labels
keywords = pd.read_csv(data_path+descriptions_labels_keys_file)

In [ ]:
# There is a missing key
keywords.isna().sum()

key            1
cluster        0
description    0
label          0
new_label      0
dtype: int64

In [ ]:
# Fill in the missing value
keywords["key"] = keywords["key"].fillna("no key")

# Create index for doi numbers - this is needed because many articles are missing doi number
keywords['key_id'] = keywords['key'].apply(hash_text_md5)

# Save keywords to a file
keywords.to_csv(keywords_file, index=False)

# Check for success
keywords.head(1)

,key,cluster,description,label,new_label,key_id
0,uncountably many different asymptotic growth r...,0,"The study of how populations grow, decline, an...",Population Dynamics,Population Dynamics_0,a87e17d3c9befeb236f46bf77da29d6d


### Parse the Articles

In [ ]:
# Articles, titles, abstracts and keywords
articles = pd.read_csv(data_path+parsed_keys_file)

In [ ]:
# The keywords column entries must be a list
articles['abstracts_keys'] = articles['abstracts_keys'].apply(ast.literal_eval)
articles['authors_parsed'] = articles['authors_parsed'].apply(ast.literal_eval)
articles['versions'] = articles['versions'].apply(ast.literal_eval)

In [ ]:
# Rename columns
articles.rename(columns={'journal-ref': 'journal', 'report-no': 'report', 'id': 'article_id'}, inplace=True)

In [ ]:
# Identify the missing entries
articles.isna().sum()

journal           1347
authors_parsed       0
versions             0
title                0
comments           841
article_id           0
update_date          0
categories           0
license           5966
abstract             0
doi                  0
submitter           47
report            4981
abstracts_keys       0
dtype: int64

In [ ]:
# Address the missing values
articles["comments"] = articles["comments"].fillna("no comments")
articles["license"] = articles["license"].fillna("not specified")
articles["journal"] = articles["journal"].fillna("no journal")
articles["submitter"] = articles["submitter"].fillna("no submitter")
articles["report"] = articles["report"].fillna("none provided")

# Check for success
articles.head()

,journal,authors_parsed,versions,title,comments,article_id,update_date,categories,license,abstract,doi,submitter,report,abstracts_keys
0,"Constructive Approximation 18 (2002), 479-502","[[Warnaar, S. O., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...",Summation and transformation formulas for elli...,"21 pages, AMS-LaTeX",1006,2010-06-18,math.QA math.CO,not specified,Using matrix inversion and determinant evalu...,10.1007/s00365-002-0501-6,S. Ole Warnaar,ITF-99-42,"[hypergeometric, summation, elliptic, determin..."
1,Commun.Math.Phys. 213 (2000) 641-672,"[[Dunajski, Maciej, ], [Mason, Lionel J., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...","Hyper-K{\""a}hler Hierarchies and their twistor...","23 pages, 1 figure",1008,2009-10-31,math.DG gr-qc nlin.SI,not specified,A twistor construction of the hierarchy asso...,10.1007/PL00005532,Maciej Dunajski,none provided,"[symmetry algebra, recursion operator, hyper-K..."
2,"Journal of Mathematical Logic Vol. 01, No. 02,...","[[Dougherty, Randall, , Ohio State University]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...",Solutions to congruences using sets with the p...,20 pages,1009,2021-02-09,math.MG math.LO,not specified,Hausdorff's paradoxical decomposition of a s...,10.1142/S0219061301000077,Randall Dougherty,none provided,"[isometries, homeomorphisms, partitions, congr..."
3,"Journal of Mathematical Logic Vol. 01, No. 02,...","[[Dougherty, Randall, , Ohio State University]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...",Open sets satisfying systems of congruences,44 pages,1010,2021-02-09,math.MG,not specified,A famous result of Hausdorff states that a s...,10.1142/S0219061301000107,Randall Dougherty,none provided,"[isometries, sphere, partitions, congruences, ..."
4,"J. Number Theory 87 (2001), no. 2, 253-269","[[Burnol, Jean-Francois, , Nice - Sophia Antip...","[{'version': 'v1', 'created': 'Tue, 4 Jan 2000...",An adelic causality problem related to abelian...,"18 pages, latex2e with amsfonts. Final version...",1013,2007-05-23,math.NT,not specified,I associate to a global field K a Lax-Philli...,10.1006/jnth.2000.2616,Jean-Francois Burnol,PUMA 566,"[lax-phillips scattering, causality, abelian l..."


### Parse the Authors

In [ ]:
# Create a list of all authors
df_authors = articles.authors_parsed.tolist()

# Flatten the list of sublists
flat_authors = [item for sublist in df_authors for item in sublist]

# Create a dataframe with the distinct keywords
authors_df = pd.DataFrame(flat_authors, columns = ['last_name','first_name', 'middle_name', 'affiliation'])

# Replace None or empty string in 'affiliation' with 'unspecified'
authors_df['affiliation'] = authors_df['affiliation'].replace('', 'unspecified').fillna('unspecified')

# Drop the duplicates
authors_df.drop_duplicates(inplace=True)
authors_df.reset_index(drop=True, inplace=True)

# Create an indexing id
authors_df['author_id'] = authors_df.apply(lambda row: hash_text_md5(row['last_name'] + row['first_name'] + row['affiliation']), axis=1)

# Check the output
authors_df.head()

,last_name,first_name,middle_name,affiliation,author_id
0,Warnaar,S. O.,,unspecified,fd6fcf9374c7e5fd0377800da5e8f846
1,Dunajski,Maciej,,unspecified,befdd60b44b9cd4d5ffc084ce31d92d4
2,Mason,Lionel J.,,unspecified,a95ff40457bf3cae9c6852c6a14937fa
3,Dougherty,Randall,,Ohio State University,d83c43e5b1cf398c4e549843f497694b
4,Burnol,Jean-Francois,,Nice - Sophia Antipolis,700c655b31f9b9275ba10a9875c15332


In [ ]:
# Check for missing values
authors_df.isna().sum()

last_name      0
first_name     0
middle_name    0
affiliation    0
author_id      0
dtype: int64

In [ ]:
def get_author_id(author_details):
    """Create authors tuples and their ids."""

    author_tuple = tuple(author_details) if len(author_details) == 4 else tuple(author_details + ['unspecified'])

    # Debug: Print out the author_tuple to check what is being matched
    #print("Matching Author Tuple:", author_tuple)

    matching_authors = authors_df[(authors_df['last_name'] == author_tuple[0]) &
                                  (authors_df['first_name'] == author_tuple[1]) &
                                  (authors_df['middle_name'] == author_tuple[2]) &
                                  (authors_df['affiliation'] == author_tuple[3])]

    # Debug: Print out the matching authors
    #print("Matching Authors in DataFrame:\n", matching_authors)

    if not matching_authors.empty:
        return matching_authors.iloc[0]['author_id']
    else:
        return None

In [ ]:
# Create relationship dataframe - between authors and titles
relations = []
for index, row in articles.iterrows():
    article_id = row['article_id']
    for author in row['authors_parsed']:
        author_id = get_author_id(author)
        if author_id is not None:
            relations.append({'article_id': article_id, 'author_id': author_id})

df_relations = pd.DataFrame(relations)

In [ ]:
# Check the output
df_relations.head()

,article_id,author_id
0,1006,fd6fcf9374c7e5fd0377800da5e8f846
1,1008,befdd60b44b9cd4d5ffc084ce31d92d4
2,1008,a95ff40457bf3cae9c6852c6a14937fa
3,1009,d83c43e5b1cf398c4e549843f497694b
4,1010,d83c43e5b1cf398c4e549843f497694b


In [ ]:
# Save parsed authors to a csv file
authors_df.to_csv(data_path+authors_file, index=False)
# Save articles-titles relations to a file
df_relations.to_csv(data_path+articles_authors_relations_file, index=False)

### Parse the Categories

In [ ]:
# Parse the categories column
articles["parsed_categories"] = articles.categories.str.split(" ")
articles.head(2)

,journal,authors_parsed,versions,title,comments,article_id,update_date,categories,license,abstract,doi,submitter,report,abstracts_keys,parsed_categories
0,"Constructive Approximation 18 (2002), 479-502","[[Warnaar, S. O., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...",Summation and transformation formulas for elli...,"21 pages, AMS-LaTeX",1006,2010-06-18,math.QA math.CO,not specified,Using matrix inversion and determinant evalu...,10.1007/s00365-002-0501-6,S. Ole Warnaar,ITF-99-42,"[hypergeometric, summation, elliptic, determin...","[math.QA, math.CO]"
1,Commun.Math.Phys. 213 (2000) 641-672,"[[Dunajski, Maciej, ], [Mason, Lionel J., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...","Hyper-K{\""a}hler Hierarchies and their twistor...","23 pages, 1 figure",1008,2009-10-31,math.DG gr-qc nlin.SI,not specified,A twistor construction of the hierarchy asso...,10.1007/PL00005532,Maciej Dunajski,none provided,"[symmetry algebra, recursion operator, hyper-K...","[math.DG, gr-qc, nlin.SI]"


In [ ]:

# Create a list of all sublists of categories
df_categories = articles.parsed_categories.tolist()

# Flatten the list of sublists
flat_categories = [item for sublist in df_categories for item in sublist]

# Create a list of unique keywords
flat_categories = list(set(flat_categories))

# Create a dataframe with the distinct keywords
categories_df = pd.DataFrame(flat_categories, columns = ['categories'])

# Add indexing to the categories
categories_df['category_id'] = categories_df['categories'].apply(hash_text_md5)

# Count how many distinct keywords were extracted
categories_df.head()

,categories,category_id
0,gr-qc,a717ed61a226a8a94bc8369e4f0c5fae
1,math.ST,6dfc5bd19d7f766759a2787d8c41b9b1
2,hep-th,c3e29f6d694dd121e24f3b292beafc0b
3,q-bio.MN,79cc162137e4709c2c1d0b3e66a9f186
4,physics.comp-ph,5b1975e2345bbe5e8ef823c054daa659


In [ ]:
# Determine the number of categories extracted
categories_df.shape

(109, 2)

In [ ]:
# Save the categories to a file
categories_df.to_csv(data_path+categories_file, index=False)

### Parse DOI numbers

In [ ]:
# Create a list of all doi values
df_doi = articles.doi.unique()

# Create a dataframe with the distinct doi numbers
doi_df = pd.DataFrame(df_doi, columns = ['doi'])

# Create index for doi numbers - this is needed because many articles are missing doi number
doi_df['doi_id'] = doi_df['doi'].apply(hash_text_md5)

# Save the doi info to a file
doi_df.to_csv(data_path+doi_file, index=False)

# Count how many distinct keywords were extracted
doi_df.head()

,doi,doi_id
0,10.1007/s00365-002-0501-6,57aa19619324dcd39039c52e67002af3
1,10.1007/PL00005532,ee4af8f04acefe1036a78a4436691f55
2,10.1142/S0219061301000077,fe8768ee88f2d27ed51861639e63a4ff
3,10.1142/S0219061301000107,01c710839626aa78f11824682bccef1b
4,10.1006/jnth.2000.2616,563c3ceee67bbf5fc29c764b86a8c8ca


### Parse Report Information

In [ ]:
# Create a list of all doi values
df_report = articles.report.unique()

# Create a dataframe with the distinct doi numbers
report_df = pd.DataFrame(df_report, columns = ['report'])

# Create index for doi numbers - this is needed because many articles are missing doi number
report_df['report_id'] = report_df['report'].apply(hash_text_md5)

# Save the report info to a file
report_df.to_csv(data_path+reports_parsed, index=False)

# Count how many distinct keywords were extracted
report_df.head()

,report,report_id
0,ITF-99-42,6949743cea68a5e6f3f5bd286694d356
1,none provided,dd0a54fea06e7b7a384741aac9313d65
2,PUMA 566,d4a4409b7e8a77f4894c998a04162257
3,SISSA: 39/99/FM/GEO,c3c968646b7780fada2a98b2a5a61746
4,"HU-IAS/K-8, DPSU-99-8, RIMS-1266",42a4c65c8ac88e75fed4b8e25a71dc9a


### Parse UpdateDate

In [ ]:
# Create a list of all doi values
df_udate = articles.update_date.unique()

# Create a dataframe with the distinct doi numbers
udate_df = pd.DataFrame(df_udate, columns = ['update_date'])

# Create index for doi numbers - this is needed because many articles are missing doi number
udate_df['update_id'] = udate_df['update_date'].apply(hash_text_md5)

# Save the jupdated dates to a file
udate_df.to_csv(data_path+udates_file, index=False)

# Count how many distinct keywords were extracted
udate_df.head()# Save the journal names to a file

,update_date,update_id
0,2010-06-18,b4ccd8c26a79a78f697e6485f939921a
1,2009-10-31,3a5237e3ba63387c2dfac962f4dabba4
2,2021-02-09,d9985bd71eb14eabcca5823d0f0e411d
3,2007-05-23,4521a3bff2923c2803b9cea4a21da177
4,2017-07-11,5d10df0da9f2b467da310afa18c15035


### Parse Journal

In [ ]:
def extract_journal_info(row):
    """Function to parse journal references."""

    if row == 'no journal':
        return '', '', '', ''

    # Regex patterns
    year_regex = r'(\d{4})'
    pages_regex = r'(\d+-\d+|pp\. \d+-\d+)'
    journal_meta_regex = r'(\d+|[Vv]ol\. \d+, [Nn]o\. \d+|[nN]o\. \d+)'
    journal_name_regex = r'^[A-Za-z.]+[\sA-Za-z.]*'

    # Extract data using regex
    year = re.search(year_regex, row)
    pages = re.search(pages_regex, row)
    journal_meta = re.search(journal_meta_regex, row)
    journal_name = re.search(journal_name_regex, row)

    # Process extracted data
    year = year.group() if year else None
    pages = pages.group().replace('pp. ', '') if pages else None
    journal_meta = journal_meta.group() if journal_meta else None
    if journal_name:
        journal_name = journal_name.group().rstrip(' .')
        # Remove trailing " Vol"
        journal_name = re.sub(r'\sVol$', '', journal_name)
    else:
        journal_name = "no journal"

    return journal_name, pages, year, journal_meta


In [ ]:
# Apply extraction function
articles[['journal_name',
          'pages',
          'year',
          'journal_meta']] = articles['journal'].apply(lambda x: pd.Series(extract_journal_info(x)))
articles.head(2)

,journal,authors_parsed,versions,title,comments,article_id,update_date,categories,license,abstract,doi,submitter,report,abstracts_keys,parsed_categories,journal_name,pages,year,journal_meta
0,"Constructive Approximation 18 (2002), 479-502","[[Warnaar, S. O., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...",Summation and transformation formulas for elli...,"21 pages, AMS-LaTeX",1006,2010-06-18,math.QA math.CO,not specified,Using matrix inversion and determinant evalu...,10.1007/s00365-002-0501-6,S. Ole Warnaar,ITF-99-42,"[hypergeometric, summation, elliptic, determin...","[math.QA, math.CO]",Constructive Approximation,479-502,2002,18
1,Commun.Math.Phys. 213 (2000) 641-672,"[[Dunajski, Maciej, ], [Mason, Lionel J., ]]","[{'version': 'v1', 'created': 'Mon, 3 Jan 2000...","Hyper-K{\""a}hler Hierarchies and their twistor...","23 pages, 1 figure",1008,2009-10-31,math.DG gr-qc nlin.SI,not specified,A twistor construction of the hierarchy asso...,10.1007/PL00005532,Maciej Dunajski,none provided,"[symmetry algebra, recursion operator, hyper-K...","[math.DG, gr-qc, nlin.SI]",Commun.Math.Phys,641-672,2000,213


In [ ]:

# Create a list of all journals
df_journal = articles.journal_name.unique()

# Create a dataframe with the distinct journals
journal_df = pd.DataFrame(df_journal, columns = ['journal'])

# Create id for journal names
journal_df['journal_id'] = journal_df['journal'].apply(hash_text_md5)

# The journal names needs extra parsing but we'll leave them as they are for this experiment
# Save the journal names to a file
journal_df.to_csv(data_path+journals_parsed, index=False)

# Count how many distinct keywords were extracted
journal_df.head(10)

,journal,journal_id
0,Constructive Approximation,6dc95119d8a737d9ffc7fe367159849b
1,Commun.Math.Phys,ebe5bed51965553359ee9f13db0293e1
2,Journal of Mathematical Logic,f663b7749e96288af8628ca3cdb4f021
3,J. Number Theory,f762cb2c3b5bd7f0b3941598cfce8f15
4,,d41d8cd98f00b204e9800998ecf8427e
5,Topology,99571eecaa5c5d622bb0d1f855752d5f
6,Geom. Topol,79942321e19994e38ad2e372af21c893
7,J.Statist.Phys,f6582df4742923aacfc6e67638bd4bf6
8,Math. Ann,e7b0a9b48142710ab6e87f136f6b7606
9,Math. Nachr,d3b321ed42589ea798c241aa7de23d2d


## Installs & Imports for Graph Building

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.16.0-py3-none-any.whl size=273811 sha256=7631d10efbb3508f70417a3db640b65db5d76d2ea01f09693af1894f46b0a7a6
  Stored in directory: /root/.cache/pip/wheels/20/a0/f6/87a1ec9636c915fe2d6c6e859fd55a6231dd9bc95a1d5394b1
Successfully built neo4j


In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import hashlib
from neo4j import time


## Establish Neo 4j Connection

In [ ]:
# Import the Neo4j connector module
from utils.neo4j_conn import *

In [ ]:
# Create an Neo4j AuraDB free instance and collect the credentials
URI = 'neo4j+s://xxxxxxxx.databases.neo4j.io'
USER = 'neo4j'
PWD = 'your_password_here'

# Initialize the Neo4j connector
graph=Neo4jGraph(url=URI, username=USER, password=PWD)

In [ ]:
# Check the connection
graph.query("MATCH (n) RETURN count(n)")

[{'count(n)': 38650}]

## Build the Knowledge Graph

In [ ]:
# Create uniqueness constraints on each node

constraints_articles = """
CREATE CONSTRAINT article_id IF NOT EXISTS FOR (article:Article) REQUIRE article.id IS UNIQUE;"""
graph.query(constraints_articles)

constraints_keywords="""
CREATE CONSTRAINT keyword_key IF NOT EXISTS FOR (keyword:Keyword) REQUIRE keyword.key IS UNIQUE;
"""
graph.query(constraints_keywords)

constraints_clusters="""
CREATE CONSTRAINT topic_cluster IF NOT EXISTS FOR (topic:Topic) REQUIRE topic.cluster IS UNIQUE;
"""
graph.query(constraints_clusters)

constraints_authors="""
CREATE CONSTRAINT author IF NOT EXISTS FOR (author:Author) REQUIRE author.author_id IS UNIQUE;
"""
graph.query(constraints_authors)

constraints_categories="""
CREATE CONSTRAINT category IF NOT EXISTS FOR (category:Categories) REQUIRE category.category_id IS UNIQUE;
"""
graph.query(constraints_categories)

constraints_doi="""
CREATE CONSTRAINT doi IF NOT EXISTS FOR (doi:DOI) REQUIRE doi.doi_id IS UNIQUE;
"""
graph.query(constraints_doi)

constraints_reports="""
CREATE CONSTRAINT report IF NOT EXISTS FOR (report:Report) REQUIRE report.report_id IS UNIQUE;
"""
graph.query(constraints_reports)

constraints_udates="""
CREATE CONSTRAINT udate IF NOT EXISTS FOR (udate:UpdateDate) REQUIRE udate.udate_id IS UNIQUE;
"""
graph.query(constraints_udates)

constraints_journals="""
CREATE CONSTRAINT journal IF NOT EXISTS FOR (journal:Journal) REQUIRE journal.journal_id IS UNIQUE;
"""
graph.query(constraints_journals)

[]

In [ ]:
# Load Keyword and Topic nodes, and the relationship HAS_TOPIC
query_keywords_topics = """
    UNWIND $rows AS row
    MERGE (k:Keyword {key_id: row.key_id, name: row.key})
    MERGE (t:Topic {cluster: row.cluster, description: row.description, label: row.new_label})
    MERGE (k)-[:HAS_TOPIC]->(t)
    """
graph.load_data(query_keywords_topics, keywords)

In [ ]:
# Set the cluster label to INTEGER
query_dt = """
MATCH (t:Topic)
SET t.cluster = toInteger(t.cluster)
"""
graph.query(query_dt)

[]

In [ ]:
# Load doi nodes
query_doi = """
    UNWIND $rows AS row
    MERGE (d:DOI {doi_id: row.doi_id, name: row.doi})
    """
graph.load_data(query_doi, doi_df)

In [ ]:
# Load categories nodes
query_categories = """
    UNWIND $rows AS row
    MERGE (c:Categories {category_id: row.category_id, specifications: row.categories})
    """
graph.load_data(query_categories, categories_df)

In [ ]:
# Load report nodes
query_reports = """
    UNWIND $rows AS row
    MERGE (r:Report {report_id: row.report_id, report_no: row.report})
    """
graph.load_data(query_reports, report_df)

In [ ]:
# Load Article-Report relationship HAS_REPORT
query_articles_reports = """
    UNWIND $rows as row
    MERGE (a:Article {article_id: row.article_id})
    WITH a, row
    MATCH (r:Report {report_no: row.report})
    MERGE (a)-[:HAS_REPORT]-> (r)
    """
graph.load_data(query_articles_reports, articles)

In [ ]:
# Add dates
query_dates = """
UNWIND $rows AS row
MERGE (d:UpdateDate {update_date: row.update_date})
WITH d, row
MATCH (a:Article {article_id: row.article_id})
MERGE (a)-[:UPDATED]->(d)
"""
graph.load_data(query_dates, articles)

In [ ]:
# Set the udate to date
query_ud = """
MATCH (u:UpdateDate)
SET u.update_date = date(u.update_date)
"""
graph.query(query_ud)

[]

In [ ]:
# Load Article nodes and the relationship HAS_KEY
query_articles = """
    UNWIND $rows as row
    MERGE (a:Article {article_id: row.article_id, title: row.title, abstract: row.abstract, comments: row.comments})

    WITH a, row
    UNWIND row.abstracts_keys as key
    MATCH (k:Keyword {name: key})
    MERGE (a)-[:HAS_KEY]->(k)

    WITH a, row
    MATCH (d:DOI {name: row.doi})
    MERGE (a)-[:HAS_DOI]-> (d)

    WITH a, row
    UNWIND row.parsed_categories as category
    MATCH (c:Categories {specifications: category})
    MERGE (a)-[:HAS_CATEGORY]->(c)
    """
graph.load_data(query_articles, articles)

In [ ]:
# Load Authors
query_authors = """
    UNWIND $rows AS row
    MERGE (a:Author {author_id: row.author_id, last_name: row.last_name, first_name:row.first_name, affiliation: row.affiliation})
    """
graph.load_data(query_authors, authors_df)

In [ ]:
# Load relationships between Articles and Authors
query_rels = """
    UNWIND $rows AS row
    MATCH (a:Article {article_id: row.article_id})
    MATCH (au:Author {author_id: row.author_id})
    MERGE (a)-[:WRITTEN_BY]->(au)
    """
graph.load_data(query_rels, df_relations)

In [ ]:
# Create Journal nodes
query_journals = """
UNWIND $rows as row
MERGE (j:Journal {journal_id:row.journal_id, name: row.journal})
"""
graph.load_data(query_journals, journal_df)

In [ ]:
# Check for missing values
articles.isna().sum()

journal             0
authors_parsed      0
versions            0
title               0
comments            0
article_id          0
update_date         0
categories          0
license             0
abstract            0
doi                 0
submitter           0
report              0
abstracts_keys      0
journal_name        0
pages             895
year                3
journal_meta        0
dtype: int64

In [ ]:
# Replace None or empty string in 'affiliation' with 'unspecified'
articles['pages'] = articles['pages'].fillna('unspecified')
articles['year'] = articles['year'].fillna('1000')

In [ ]:
# Connect Journals with Articles
query_published = """
UNWIND $rows AS row
MATCH (a:Article {article_id: row.article_id})
WITH a, row
MATCH (j:Journal {name: row.journal_name})
MERGE (a)-[r:PUBLISHED_IN {pages: row.pages, year: row.year, meta:row.journal_meta}]->(j)
"""

graph.load_data(query_published, articles)

## Sample Queries

In [ ]:
# Query to extract the graph schema
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""
node_props = graph.query(node_properties_query)
node_props

[{'output': {'labels': 'Article',
   'properties': ['abstract', 'article_id', 'comments', 'title']}},
 {'output': {'labels': 'Keyword', 'properties': ['name', 'key_id']}},
 {'output': {'labels': 'Topic',
   'properties': ['cluster', 'description', 'label']}},
 {'output': {'labels': 'Author',
   'properties': ['author_id', 'affiliation', 'first_name', 'last_name']}},
 {'output': {'labels': 'DOI', 'properties': ['name', 'doi_id']}},
 {'output': {'labels': 'Categories',
   'properties': ['category_id', 'specifications']}},
 {'output': {'labels': 'Report', 'properties': ['report_id', 'report_no']}},
 {'output': {'labels': 'UpdateDate', 'properties': ['update_date']}},
 {'output': {'labels': 'Journal', 'properties': ['name', 'journal_id']}}]

In [ ]:
# Query to extract relationships list
rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""
rels = graph.query(rel_query)
rels

[{'output': {'relationship': 'HAS_KEY',
   'source': 'Article',
   'target': ['Keyword']}},
 {'output': {'relationship': 'HAS_DOI',
   'source': 'Article',
   'target': ['DOI']}},
 {'output': {'relationship': 'HAS_CATEGORY',
   'source': 'Article',
   'target': ['Categories']}},
 {'output': {'relationship': 'WRITTEN_BY',
   'source': 'Article',
   'target': ['Author']}},
 {'output': {'relationship': 'UPDATED',
   'source': 'Article',
   'target': ['UpdateDate']}},
 {'output': {'relationship': 'PUBLISHED_IN',
   'source': 'Article',
   'target': ['Journal']}},
 {'output': {'relationship': 'HAS_REPORT',
   'source': 'Article',
   'target': ['Report']}},
 {'output': {'relationship': 'HAS_TOPIC',
   'source': 'Keyword',
   'target': ['Topic']}}]

In [ ]:
# Find 5 articles that contain algebra in the title and abstract

read_query = """
MATCH (a:Article)
WHERE a.abstract CONTAINS 'algebra' AND a.title CONTAINS 'algebra'
RETURN a.title as Title, a.abstract AS Abstract
LIMIT 5
"""
graph.query(read_query)

[{'Title': 'The Gervais-Neveu-Felder equation for the Jordanian quasi-Hopf\n  U_{h;y}(sl(2)) algebra',
  'Abstract': '  Using a contraction procedure, we construct a twist operator that satisfies a\nshifted cocycle condition, and leads to the Jordanian quasi-Hopf U_{h;y}(sl(2))\nalgebra. The corresponding universal ${\\cal R}_{h}(y)$ matrix obeys a\nGervais-Neveu-Felder equation associated with the U_{h;y}(sl(2)) algebra. For a\nclass of representations, the dynamical Yang-Baxter equation may be expressed\nas a compatibility condition for the algebra of the Lax operators.\n'},
 {'Title': 'Twist deformations for generalized Heisenberg algebras',
  'Abstract': '  Multidimensional Heisenberg algebras, whose creation and annihilation\noperators are the N-dimensional vectors, can be injected into simple Lie\nalgebras g. It is demonstrated that the spectrum of their deformations can be\ninvestigated using chains of extended Jordanian twists applied to U(g). In the\ncase of U(sl(N)) (for N>5)

In [ ]:
# Basic node retrieval
# Fetch 5 journals in the database

query = """
MATCH (j:Journal)
RETURN j.name LIMIT 5
"""

graph.query(query)

[{'j.name': 'Rev. Mat.Iberoamericana'},
 {'j.name': 'Finite Fields Appl'},
 {'j.name': 'J. Geom. Analysis'},
 {'j.name': 'Comm. Partial Differential Equations'},
 {'j.name': 'Proyecciones'}]

In [ ]:
# Find the most published author

read_query="""
MATCH (a:Author)-[]-(p:Article)-[]-(j:Journal)
RETURN a.last_name as LastName, a.first_name AS FirstNAme, count(p) as Freq
ORDER BY Freq DESC
LIMIT 5
"""
graph.query(read_query)

[{'LastName': 'Schick', 'FirstNAme': 'Thomas', 'Freq': 27},
 {'LastName': 'Bartholdi', 'FirstNAme': 'Laurent', 'Freq': 23},
 {'LastName': 'Kotschick', 'FirstNAme': 'D.', 'Freq': 20},
 {'LastName': 'Chakrabarti', 'FirstNAme': 'A.', 'Freq': 18},
 {'LastName': 'Suciu', 'FirstNAme': 'Alexander I.', 'Freq': 18}]

In [ ]:
# Node retrieval with property filtering
# Fetch articles published after a specific date

query = """
MATCH (a:Article)-[]-(ud:UpdateDate)
WHERE date(ud.update_date).year = 2007
RETURN a.title, ud.update_date
LIMIT 4
"""

graph.query(query)

[{'a.title': 'Reconstruction of Gray-scale Images',
  'ud.update_date': neo4j.time.Date(2007, 7, 2)},
 {'a.title': 'Finite-Dimensional Crystals B^{2,s} for Quantum Affine Algebras of type\n  D_{n}^{(1)}',
  'ud.update_date': neo4j.time.Date(2007, 10, 8)},
 {'a.title': 'On nonparametric maximum likelihood for a class of stochastic inverse\n  problems',
  'ud.update_date': neo4j.time.Date(2007, 10, 8)},
 {'a.title': 'On the strong consistency of asymptotic M-estimators',
  'ud.update_date': neo4j.time.Date(2007, 10, 8)}]

In [ ]:
# Fetch 10 articles and their authors published in a specific journal

query = """
MATCH (j:Journal {name: "Commun.Math.Phys"})<-[:PUBLISHED_IN]-(a:Article)-[:WRITTEN_BY]-(au:Author)
RETURN a.title, COLLECT(au.last_name + ', ' + au.first_name) AS authors
LIMIT 10
"""
graph.query(query)


[{'a.title': 'Hyper-K{\\"a}hler Hierarchies and their twistor theory',
  'authors': ['Dunajski, Maciej', 'Mason, Lionel J.']},
 {'a.title': '$A_{\\infty}$-structures on an elliptic curve',
  'authors': ['Polishchuk, Alexander']},
 {'a.title': 'Superselection Theory for Subsystems',
  'authors': ['Conti, Roberto', 'Doplicher, Sergio', 'Roberts, John E.']},
 {'a.title': 'Geometrical Tools for Quantum Euclidean Spaces',
  'authors': ['Cerchiai, B. L.', 'Fiore, G.', 'Madore, J.']},
 {'a.title': 'Classification of Subsystems for Local Nets with Trivial Superselection\n  Structure',
  'authors': ['Conti, Roberto', 'Carpi, Sebastiano']},
 {'a.title': 'Notes for a Quantum Index Theorem',
  'authors': ['Longo, Roberto']},
 {'a.title': 'A New Cohomology Theory for Orbifold',
  'authors': ['Chen, Weimin', 'Ruan, Yongbin']},
 {'a.title': 'Log mirror symmetry and local mirror symmetry',
  'authors': ['Takahashi, Nobuyoshi']},
 {'a.title': 'Quantum Affine (Super)Algebras $U_q(A_{1}^{(1)})$ and $U_q(

In [ ]:
# Fetch all authors who wrote a particular article

query = """
MATCH (a:Author)<-[:WRITTEN_BY]-(art:Article {article_id: 1008})
RETURN a.last_name, a.first_name
"""

graph.query(query)

[{'a.last_name': 'Dunajski', 'a.first_name': 'Maciej'},
 {'a.last_name': 'Mason', 'a.first_name': 'Lionel J.'}]

In [ ]:
# Find the journals in which an author's articles were published

query = """
MATCH path = (a:Author {last_name: "Warnaar"})-[]-(p:Article)-[]-(j:Journal)
RETURN j.name
"""
graph.query(query)

[{'j.name': 'Constructive Approximation'},
 {'j.name': 'J.Statist.Phys'},
 {'j.name': 'Discrete Mathematics'},
 {'j.name': 'Commun. Math. Phys'},
 {'j.name': ''}]

In [ ]:
# Relationships with property filtering
# Fetch articles written by a specific author and published after a certain date

query= """
MATCH (a:Author {last_name: "Schick"})-[]-(art:Article)-[]-(ud:UpdateDate)
WHERE ud.update_date > "2000-01-01"
RETURN art.title, ud.update_date
"""
graph.query(query)

[{'art.title': "A K-Theoretic Proof of Boutet de Monvel's Index Theorem for Boundary\n  Value Problems",
  'ud.update_date': '2007-05-23'},
 {'art.title': 'Finite group extensions and the Baum-Connes conjecture',
  'ud.update_date': '2014-11-11'},
 {'art.title': 'On a conjecture of Atiyah', 'ud.update_date': '2015-06-26'},
 {'art.title': 'Integrality of L2-Betti numbers',
  'ud.update_date': '2018-11-28'},
 {'art.title': 'Manifolds with boundary and of bounded geometry',
  'ud.update_date': '2018-11-28'},
 {'art.title': 'Approximating L2-invariants, and the Atiyah conjecture',
  'ud.update_date': '2018-11-28'},
 {'art.title': 'The spectral measure of certain elements of the complex group ring of a\n  wreath product',
  'ud.update_date': '2018-11-28'},
 {'art.title': 'Approximating L^2-signatures by their compact analogues',
  'ud.update_date': '2018-11-28'},
 {'art.title': 'Approximating Spectral invariants of Harper operators on graphs II',
  'ud.update_date': '2018-11-28'},
 {'art.ti

In [ ]:
# Multiple paths
# Find authors who have written articles for a specific journal

query = """
MATCH (a:Author)-[]-(:Article)-[]-(j:Journal)
WHERE j.name CONTAINS "Comm"
RETURN DISTINCT a.last_name AS Name
LIMIT 10
"""
graph.query(query)

[{'Name': 'Gioev'},
 {'Name': 'Coriasco'},
 {'Name': 'Schrohe'},
 {'Name': 'Seiler'},
 {'Name': 'Barles'},
 {'Name': 'Ley'},
 {'Name': 'Mangoubi'},
 {'Name': 'Dunajski'},
 {'Name': 'Mason'},
 {'Name': 'Polishchuk'}]

In [ ]:
# Combining Aggregations and Paths
# Find the journal that has published the most articles:

query = """
    MATCH (j:Journal)-[]-(a:Article)
    WHERE j.name <> ''
    RETURN j.name AS Journal, COUNT(a) AS NumberArticles
    ORDER BY NumberArticles DESC
    LIMIT 2
    """
graph.query(query)

[{'Journal': 'Algebr. Geom. Topol', 'NumberArticles': 344},
 {'Journal': 'Geom. Topol', 'NumberArticles': 285}]

In [ ]:
# Complex Aggregations with Filtering
# Find authors who have written more than 5 articles and at
# least one of those articles was published in the "Topology" journal:

query = """
MATCH (a:Author)<-[:WRITTEN_BY]-(art:Article)
WITH a, COUNT(art) AS ArticleCount
WHERE ArticleCount > 5
MATCH (a)<-[:WRITTEN_BY]-(:Article)-[:PUBLISHED_IN]->(j:Journal)
WHERE j.name CONTAINS 'Topology'
RETURN a.last_name AS LastName, a.first_name AS FirstName, ArticleCount, j.name AS Journal
"""
pd.DataFrame(graph.query(query))

,LastName,FirstName,ArticleCount,Journal
0,Christensen,J. Daniel,7,Topology
1,Christensen,J. Daniel,7,Topology
2,Suciu,Alexander I.,18,Topology
3,Suciu,Alexander I.,18,Topology and Appl
4,Kotschick,D.,20,Topology
5,Feehan,Paul M. N.,7,Topology and its Applications
6,Meyer,Ralf,13,Topology
7,Meyer,Ralf,13,Topology
8,Tsaban,Boaz,17,Topology and its Applications
9,Tsaban,Boaz,17,Topology and its Applications
